In [1]:
import tensorflow as tf
import numpy as np

/home/jlin/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


Let's play around with variables in tensorflow!

In [2]:
# use tf.constant for constants
x = tf.constant(100, name = 'x')
y = tf.constant(200, name = 'x')

# use tf.Variable for things that are variables (can change)
dist = tf.Variable(abs(x - y), name = 'dist')

# create your global variables initializer
init = tf.global_variables_initializer()

with tf.Session() as sess:
    sess.run(init)
    print(sess.run(dist))

100


Now you know how to create variables, constants and the global variables initializer. You also know how to create a session, initialize your global variables, and run a variable

Writing a Tensorflow program consists of creating variables and defining operations between them, initializing your variables, then creating and running your session on your variables.

Now let's play with placeholders. You can create placeholders with `tf.placeholder`. When you run operations with a placeholder, you need a feed dictionary, which contains the values of those placeholders. Let's write the code above with placeholders now.

In [3]:
x = tf.placeholder(tf.float32, name = 'x')
y = tf.placeholder(tf.float32, name = 'y')

d = abs(x - y)

init = tf.global_variables_initializer()

xval = 100
yval = 200
with tf.Session() as sess:
    sess.run(init)
    print(sess.run(d, feed_dict = {x: xval, y : yval}))

100.0


Let's write the sigmoid function with tensorflow

In [4]:
def sigmoid(z):
    x = tf.placeholder(tf.float32, name = 'x')
    
    sigmoid = tf.sigmoid(x)
    with tf.Session() as sess:
        a = sess.run(sigmoid, feed_dict = {x: z})
    
    return a

Now let's test our sigmoid function

In [5]:
print(sigmoid(0))
print(sigmoid(float('inf')))
print(sigmoid(float('-inf')))

0.5
1.0
0.0


Now let's define our loss function

In [6]:
def loss(y_pred, y_true):
    Y = tf.placeholder(tf.float32, name = 'Y')
    A = tf.placeholder(tf.float32, name = 'A')
    
    loss = -(tf.multiply(Y, tf.log(A)) + tf.multiply(1 - Y, tf.log(1 - A)))
    
    with tf.Session() as sess:
        l = sess.run(loss, feed_dict = {Y: y_true, A: y_pred})
        
    return l 

In [7]:
print(-1 * np.log(0.8))
print(loss(0.8, 1))

0.2231435513142097
0.22314353


A note is that you can use `tf.nn.sigmoid_cross_entropy_with_logits(logits = , labels = )`. 

Now you should implement one-hot.

In [8]:
def one_hot(labels, C):
    C = tf.constant(C, name = 'C')
    one_hot_matrix = tf.one_hot(labels, C, axis = 0)
    with tf.Session() as sess:
        one_hot = sess.run(one_hot_matrix)
        
    return one_hot

In [9]:
labels = np.array([1, 2, 3, 1, 0])
C = 4
print(one_hot(labels, C))

[[0. 0. 0. 0. 1.]
 [1. 0. 0. 1. 0.]
 [0. 1. 0. 0. 0.]
 [0. 0. 1. 0. 0.]]


So what have we learned about so far. We've learned about `tf.sigmoid, tf.nn.sigmoid_cross_entropy_with_logits, tf.one_hot`. Now we're going to learn about `tf.ones`

In [10]:
def ones(shape):
    ones = tf.ones(shape)
    with tf.Session() as sess:
        out = sess.run(ones)
        
    return out    

Let's test our function

In [11]:
print(ones((5, 4)))

[[1. 1. 1. 1.]
 [1. 1. 1. 1.]
 [1. 1. 1. 1.]
 [1. 1. 1. 1.]
 [1. 1. 1. 1.]]


Let's first make a function for creating placeholders.

In [12]:
def create_placeholders(n_x, n_y):
    X = tf.placeholder('float', (n_x, None))
    Y = tf.placeholder('float', (n_y, None))
    return X, Y

In [16]:
X, Y = create_placeholders(100, 4)
print('X.shape = ' + str(X.shape))
print('Y.shape = ' + str(Y.shape))

(100, ?)
X.shape = (100, ?)
Y.shape = (4, ?)


Let's now create a function for initializing parameters. We're going to use a 3 layer neural network.

In [17]:
def initialize_parameters(n_x, n_y):
    W1 = tf.get_variable('W1', shape = (25, n_x), initializer = tf.contrib.layers.xavier_initializer())
    b1 = tf.get_variable('b1', shape = (25, 1))
    W2 = tf.get_variable('W2', shape = (12, 25), initializer = tf.contrib.layers.xavier_initializer())
    b2 = tf.get_variable('b2', shape = (12, 1))
    W3 = tf.get_variable('W3', shape = (n_x, 12), initializer = tf.contrib.layers.xavier_initializer())
    b3 = tf.get_variable('b3', shape = (n_y, 1))
    
    parameters = {'W1': W1, 'b1': b1, 'W2': W2, 'b2': b2, 'W3': W3, 'b3': b3}
    return parameters

In [21]:
tf.reset_default_graph()

parameters = initialize_parameters(100, 4)
for k, v in parameters.items():
    print(k + str(v.shape))

W1(25, 100)
b1(25, 1)
W2(12, 25)
b2(12, 1)
W3(100, 12)
b3(4, 1)


Now let's implement the forward propagation

In [24]:
''' X is the placeholder for the input 
    return: Z3'''
def forward_propagation(X, parameters):
    
    Z1 = tf.matmul(parameters['W1'], X) + parameters['b1']
    A1 = tf.nn.relu(Z1)
    Z2 = tf.matmul(parameters['W2'], A1) + parameters['b2']
    A2 = tf.nn.relu(Z2)
    Z3 = tf.matmul(parameters['W3'], A2) + parameters['b3']
    
    return Z3

Let's write a compute cost function. After this, we can easily write our neural network model.

In [25]:
def compute_cost(Z3, Y):
    logits = tf.transpose(Z3)
    labels = tf.transpose(Y)
    
    cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits = logits, labels = labels))
    return cost


Let's use all the functions we've written and create our neural network model.

In [ ]:
def model():
    